In [1]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from gensim.models import FastText

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tsel = pd.read_csv(r'/content/drive/MyDrive/TA/Prepro_Sinyal.csv', usecols=['Username','Text','Sinyal','text_result'])
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [5]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [6]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,admin sinyal telkomsel rumah daerah rawabunga ...


## Ektraksi Fitur TF-IDF 

In [7]:
vectors= TfidfVectorizer(ngram_range=(1,2), max_features = 1000)
vectors

TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [8]:
tfidf = vectors.fit_transform(tsel['text_result'].astype(str))
tfidf

<16991x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 116944 stored elements in Compressed Sparse Row format>

In [9]:
df_tfidf = tfidf.copy()

In [10]:
X = df_tfidf
y = tsel['Sinyal']

### SMOTE

In [11]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [13]:
X_smote, y_smote = smote.fit_resample(X,y)

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_smote,y_smote, test_size=0.2, random_state=35)

## Ekspansi Fitur

In [15]:
# Load Model trained
model_exp = FastText.load('/content/drive/MyDrive/TA/model_twt/fasttext/prepTwitter.fasttext').wv

In [16]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 1) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [17]:
feature_ef = vectors.get_feature_names()

In [18]:
# Membuat dataframe data test & train
tsel_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
tsel_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [19]:
dfexp_x_train= feature_expansion(tsel_x_train, feature_ef)

100%|██████████| 1000/1000 [00:08<00:00, 113.57it/s]


In [20]:
dfexp_x_test = feature_expansion(tsel_x_test, feature_ef)

100%|██████████| 1000/1000 [00:07<00:00, 134.89it/s]


## Klasifikasi

In [21]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = []

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [22]:
x_train_csr = sparse.csr_matrix(dfexp_x_train)
x_test_csr = sparse.csr_matrix(dfexp_x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred_exp = model.predict(x_test_csr)
y_true = y_test

# Random 5

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1654   80  167]
 [ 102 1818   35]
 [ 138    9 1783]]
              precision    recall  f1-score   support

        -1.0       0.87      0.87      0.87      1901
         0.0       0.95      0.93      0.94      1955
         1.0       0.90      0.92      0.91      1930

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9082835560566299
Recall:  0.9079419518696583
F1:  0.9080033451427246
Accuracy:  0.9082267542343588


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9310720615799527

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1574   92  235]
 [  72 1833   50]
 [  73    7 1850]]
              precision    recall  f1-score   support

        -1.0       0.92      0.83      0.87      1901
         0.0       0.95      0.94      0.94      1955
         1.0       0.87      0.96      0.91      1930

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9103056451809871
Recall:  0.9080434672121211
F1:  0.9076553914895197
Accuracy:  0.9085724161769789


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9311757931955928

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1597   77  227]
 [  87 1819   49]
 [  75   10 1845]]
              precision    recall  f1-score   support

        -1.0       0.91      0.84      0.87      1901
         0.0       0.95      0.93      0.94      1955
         1.0       0.87      0.96      0.91      1930

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9107098627298544
Recall:  0.9088258326865981
F1:  0.908602246269799
Accuracy:  0.9092637400622191


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9317487362571107

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1655   78  168]
 [ 103 1817   35]
 [ 138    9 1783]]
              precision    recall  f1-score   support

        -1.0       0.87      0.87      0.87      1901
         0.0       0.95      0.93      0.94      1955
         1.0       0.90      0.92      0.91      1930

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9083271698292688
Recall:  0.9079467951948165
F1:  0.9080195574627165
Accuracy:  0.9082267542343588


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9310753700004014

# Random 15

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1575   90  260]
 [  88 1794   48]
 [  44    4 1883]]
              precision    recall  f1-score   support

        -1.0       0.92      0.82      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.86      0.98      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9107693792619643
Recall:  0.9076193033985582
F1:  0.9068946477396332
Accuracy:  0.9077082613204286


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9307326739992571

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1660   80  185]
 [  99 1789   42]
 [ 123    9 1799]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.93      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9075353686368685
Recall:  0.9069741013256012
F1:  0.906984431634794
Accuracy:  0.9070169374351884


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.92633106977275

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1585   81  259]
 [  94 1791   45]
 [  77   10 1844]]
              precision    recall  f1-score   support

        -1.0       0.90      0.82      0.86      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.86      0.95      0.90      1931

    accuracy                           0.90      5786
   macro avg       0.90      0.90      0.90      5786
weighted avg       0.90      0.90      0.90      5786

Precision:  0.9042465906539983
Recall:  0.9021005073390076
F1:  0.9016621393474972
Accuracy:  0.9021776702385067


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9282745020129083

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1665   83  177]
 [  92 1793   45]
 [ 134   15 1782]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.92      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9059612134799675
Recall:  0.9055961722654327
F1:  0.9056190670075482
Accuracy:  0.9056342896647079


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9275211722325611

# Random 25

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1655   78  168]
 [ 103 1817   35]
 [ 138    9 1783]]
              precision    recall  f1-score   support

        -1.0       0.87      0.87      0.87      1901
         0.0       0.95      0.93      0.94      1955
         1.0       0.90      0.92      0.91      1930

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9083271698292688
Recall:  0.9079467951948165
F1:  0.9080195574627165
Accuracy:  0.9082267542343588


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9310753700004014

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1660   80  185]
 [  99 1789   42]
 [ 123    9 1799]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.93      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9075353686368685
Recall:  0.9069741013256012
F1:  0.906984431634794
Accuracy:  0.9070169374351884


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.92633106977275

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1585   81  259]
 [  94 1791   45]
 [  77   10 1844]]
              precision    recall  f1-score   support

        -1.0       0.90      0.82      0.86      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.86      0.95      0.90      1931

    accuracy                           0.90      5786
   macro avg       0.90      0.90      0.90      5786
weighted avg       0.90      0.90      0.90      5786

Precision:  0.9042465906539983
Recall:  0.9021005073390076
F1:  0.9016621393474972
Accuracy:  0.9021776702385067


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9282745020129083

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1665   83  177]
 [  92 1793   45]
 [ 134   15 1782]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.92      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9059612134799675
Recall:  0.9055961722654327
F1:  0.9056190670075482
Accuracy:  0.9056342896647079


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9275211722325611

# Random 35

### Top 1 

In [23]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1621   74  231]
 [  90 1784   44]
 [  44   11 1887]]
              precision    recall  f1-score   support

        -1.0       0.92      0.84      0.88      1926
         0.0       0.95      0.93      0.94      1918
         1.0       0.87      0.97      0.92      1942

    accuracy                           0.91      5786
   macro avg       0.92      0.91      0.91      5786
weighted avg       0.92      0.91      0.91      5786

Precision:  0.9169902727217184
Recall:  0.9144849819236138
F1:  0.9141667188296427
Accuracy:  0.914621500172831


#### ROC-AUC

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [26]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [27]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9358707794095106

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1660   80  185]
 [  99 1789   42]
 [ 123    9 1799]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.93      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9075353686368685
Recall:  0.9069741013256012
F1:  0.906984431634794
Accuracy:  0.9070169374351884


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.92633106977275

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1585   81  259]
 [  94 1791   45]
 [  77   10 1844]]
              precision    recall  f1-score   support

        -1.0       0.90      0.82      0.86      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.86      0.95      0.90      1931

    accuracy                           0.90      5786
   macro avg       0.90      0.90      0.90      5786
weighted avg       0.90      0.90      0.90      5786

Precision:  0.9042465906539983
Recall:  0.9021005073390076
F1:  0.9016621393474972
Accuracy:  0.9021776702385067


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9282745020129083

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1665   83  177]
 [  92 1793   45]
 [ 134   15 1782]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.92      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9059612134799675
Recall:  0.9055961722654327
F1:  0.9056190670075482
Accuracy:  0.9056342896647079


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9275211722325611

# Random 45

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1560   70  230]
 [  86 1875   43]
 [  75    5 1842]]
              precision    recall  f1-score   support

        -1.0       0.91      0.84      0.87      1860
         0.0       0.96      0.94      0.95      2004
         1.0       0.87      0.96      0.91      1922

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9129700619593928
Recall:  0.9109050369604184
F1:  0.9107435057905908
Accuracy:  0.91202903560318


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9335372498979634

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1570   70  220]
 [  84 1875   45]
 [  94    7 1821]]
              precision    recall  f1-score   support

        -1.0       0.90      0.84      0.87      1860
         0.0       0.96      0.94      0.95      2004
         1.0       0.87      0.95      0.91      1922

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9105617409783001
Recall:  0.9090551121136152
F1:  0.9089660274195567
Accuracy:  0.9101278949187694


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9321475317522046

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1575   64  221]
 [  96 1869   39]
 [  93   12 1817]]
              precision    recall  f1-score   support

        -1.0       0.89      0.85      0.87      1860
         0.0       0.96      0.93      0.95      2004
         1.0       0.87      0.95      0.91      1922

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9095340146200158
Recall:  0.9082594436517185
F1:  0.9081670771304577
Accuracy:  0.9092637400622191


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9315424595706623

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1665   83  177]
 [  92 1793   45]
 [ 134   15 1782]]
              precision    recall  f1-score   support

        -1.0       0.88      0.86      0.87      1925
         0.0       0.95      0.93      0.94      1930
         1.0       0.89      0.92      0.91      1931

    accuracy                           0.91      5786
   macro avg       0.91      0.91      0.91      5786
weighted avg       0.91      0.91      0.91      5786

Precision:  0.9059612134799675
Recall:  0.9055961722654327
F1:  0.9056190670075482
Accuracy:  0.9056342896647079


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(y_pred_exp.reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.9275211722325611